https://adventofcode.com/2021/day/5

<h2>--- Day 5: Hydrothermal Venture ---</h2><p>You come across a field of <a href="https://en.wikipedia.org/wiki/Hydrothermal_vent" target="_blank">hydrothermal vents</a> on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.</p>
<p>They tend to form in <em>lines</em>; the submarine helpfully produces a list of nearby <span title="Maybe they're Bresenham vents.">lines of vents</span> (your puzzle input) for you to review. For example:</p>
<pre><code>0,9 -&gt; 5,9
8,0 -&gt; 0,8
9,4 -&gt; 3,4
2,2 -&gt; 2,1
7,0 -&gt; 7,4
6,4 -&gt; 2,0
0,9 -&gt; 2,9
3,4 -&gt; 1,4
0,0 -&gt; 8,8
5,5 -&gt; 8,2
</code></pre>
<p>Each line of vents is given as a line segment in the format <code>x1,y1 -&gt; x2,y2</code> where <code>x1</code>,<code>y1</code> are the coordinates of one end the line segment and <code>x2</code>,<code>y2</code> are the coordinates of the other end. These line segments include the points at both ends. In other words:</p>
<ul>
<li>An entry like <code>1,1 -&gt; 1,3</code> covers points <code>1,1</code>, <code>1,2</code>, and <code>1,3</code>.</li>
<li>An entry like <code>9,7 -&gt; 7,7</code> covers points <code>9,7</code>, <code>8,7</code>, and <code>7,7</code>.</li>
</ul>
<p>For now, <em>only consider horizontal and vertical lines</em>: lines where either <code>x1 = x2</code> or <code>y1 = y2</code>.</p>
<p>So, the horizontal and vertical lines from the above list would produce the following diagram:</p>
<pre><code>.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
</code></pre>
<p>In this diagram, the top left corner is <code>0,0</code> and the bottom right corner is <code>9,9</code>. Each position is shown as <em>the number of lines which cover that point</em> or <code>.</code> if no line covers that point. The top-left pair of <code>1</code>s, for example, comes from <code>2,2 -&gt; 2,1</code>; the very bottom row is formed by the overlapping lines <code>0,9 -&gt; 5,9</code> and <code>0,9 -&gt; 2,9</code>.</p>
<p>To avoid the most dangerous areas, you need to determine <em>the number of points where at least two lines overlap</em>. In the above example, this is anywhere in the diagram with a <code>2</code> or larger - a total of <code><em>5</em></code> points.</p>
<p>Consider only horizontal and vertical lines. <em>At how many points do at least two lines overlap?</em></p>


In [1]:
import pandas as pd
import numpy as np

In [348]:
df = pd.read_csv("Hydrothermal.csv",sep="->",header=None, names=["p1", "p2"])

<ipython-input-348-a06cfc7d6497>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("Hydrothermal.csv",sep="->",header=None, names=["p1", "p2"])


In [349]:
sp1 = df["p1"].str.split(",",n = 1, expand = True)
sp1[:5]

,0,1
0,556,286
1,337,201
2,786,780
3,977,864
4,544,127


In [350]:
df["x1"] = sp1[0].astype(int)
df["y1"] = sp1[1].astype(int)
df.head(5)

,p1,p2,x1,y1
0,"556,286","341,71",556,286
1,"337,201","782,646",337,201
2,"786,780","117,111",786,780
3,"977,864","199,86",977,864
4,"544,127","544,144",544,127


In [351]:
sp2 = df["p2"].str.split(",",n = 1, expand = True)
df["x2"] = sp2[0].astype(int)
df["y2"] = sp2[1].astype(int)
df.head(5)

,p1,p2,x1,y1,x2,y2
0,"556,286","341,71",556,286,341,71
1,"337,201","782,646",337,201,782,646
2,"786,780","117,111",786,780,117,111
3,"977,864","199,86",977,864,199,86
4,"544,127","544,144",544,127,544,144


In [352]:
df.drop(columns =["p1", "p2"], inplace = True)
df.head(5)

,x1,y1,x2,y2
0,556,286,341,71
1,337,201,782,646
2,786,780,117,111
3,977,864,199,86
4,544,127,544,144


In [353]:
x_max = max([np.max(df["x1"]), np.max(df["x2"])])+1
y_max = max([np.max(df["y1"]), np.max(df["y2"])])+1
tablero = np.full((y_max, x_max), 0, dtype=int)
tablero

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [361]:
def marcarTablero(tablero, x1, y1, x2, y2, diagonal = False):
    x_inf = min([x1, x2])
    x_sup = max([x1, x2])

    y_inf = min([y1, y2])
    y_sup = max([y1, y2])
    
    if x1 == x2:
        tablero[:,x1][y_inf:y_sup+1]+=1
    else:
        if y1 == y2:
            tablero[y1][x_inf:x_sup+1]+=1
    if diagonal:
        puntos = determinarDiagonal(x1, y1, x2, y2)
        for p in puntos:
            tablero[p[1]][p[0]]+=1

In [355]:
for i in range(df.shape[0]):
    fila = df.iloc[i]
    marcarTablero(tablero, fila["x1"],fila["y1"],
                 fila["x2"],fila["y2"])
tablero

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [356]:
tablero[tablero[:,]>1].size

5145

### resultado  5145

<h2 id="part2">--- Part Two ---</h2><p>Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider <em>diagonal lines</em>.</p>
<p>Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:</p>
<ul>
<li>An entry like <code>1,1 -&gt; 3,3</code> covers points <code>1,1</code>, <code>2,2</code>, and <code>3,3</code>.</li>
<li>An entry like <code>9,7 -&gt; 7,9</code> covers points <code>9,7</code>, <code>8,8</code>, and <code>7,9</code>.</li>
</ul>
<p>Considering all lines from the above example would now produce the following diagram:</p>
<pre><code>1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
</code></pre>
<p>You still need to determine <em>the number of points where at least two lines overlap</em>. In the above example, this is still anywhere in the diagram with a <code>2</code> or larger - now a total of <code><em>12</em></code> points.</p>
<p>Consider all of the lines. <em>At how many points do at least two lines overlap?</em></p>


In [363]:
def determinarDiagonal(x1, y1, x2, y2):
    x_tam = abs(x2 - x1)
    y_tam = abs(y2 - y1)
    if x_tam == y_tam:
        x_inf = min([x1, x2])
        x_sup = max([x1, x2])
        
        y_inf = min([y1, y2])
        y_sup = max([y1, y2])
        if x2>x1 and y2>y1:
            return [(x1+i, y1+i) for i in range(x_tam+1)]
        if x2<x1 and y2>y1:
            return [(x1-i, y1+i) for i in range(x_tam+1)]
        if x2>x1 and y2<y1:
            return [(x1+i, y1-i) for i in range(x_tam+1)]
        if x2<x1 and y2<y1:
            return [(x1-i, y1-i) for i in range(x_tam+1)]
    return []

In [364]:
x_max = max([np.max(df["x1"]), np.max(df["x2"])])+1
y_max = max([np.max(df["y1"]), np.max(df["y2"])])+1
tablero = np.full((y_max, x_max), 0, dtype=int)
tablero

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [365]:
for i in range(df.shape[0]):
    fila = df.iloc[i]
    marcarTablero(tablero, fila["x1"],fila["y1"],
                 fila["x2"],fila["y2"], True)
tablero

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [360]:
tablero[tablero[:,]>1].size

16518

### resultado 16518